In [5]:
pip install capytaine   

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\akash\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [1]:
import capytaine as cpt
import logging
import vtk
import numpy as np 
logging.basicConfig(level=logging.INFO)

In [3]:
mesh = cpt.load_mesh('buoy6.dat')
#mesh = meshio.read('rack spur rectangular_am_edit.stl')
#mesh = cpt.mesh_vertical_cylinder(length=0.15, radius=0.05,center=(0, 0, 0.075),resolution=(6, 6, 8))
body = cpt.FloatingBody(mesh = mesh,
                        dofs = cpt.rigid_body_dofs(rotation_center = (0, 0, 0)),
                        center_of_mass = np.array([0, 0, 0]))

#body.mesh.vertices = 1/1000 * body.mesh.vertices

INFO:capytaine.bodies.bodies:The rotation dof Roll has been initialized around the point: buoy6.dat.rotation_center = [0. 0. 0.]
INFO:capytaine.bodies.bodies:The rotation dof Pitch has been initialized around the point: buoy6.dat.rotation_center = [0. 0. 0.]
INFO:capytaine.bodies.bodies:The rotation dof Yaw has been initialized around the point: buoy6.dat.rotation_center = [0. 0. 0.]
INFO:capytaine.bodies.bodies:New floating body: buoy6.dat.


In [4]:
mesh.show()

File 'screenshot.png' written in c:\Users\akash\OneDrive\Documents\GitHub\Nemoh\matlab routines\buoy6\mesh
File 'screenshot.png' written in c:\Users\akash\OneDrive\Documents\GitHub\Nemoh\matlab routines\buoy6\mesh
File 'screenshot.png' written in c:\Users\akash\OneDrive\Documents\GitHub\Nemoh\matlab routines\buoy6\mesh


In [5]:
hydrostatics = body.compute_hydrostatics(rho = 1025.0)
from capytaine.io.legacy import export_hydrostatics
export_hydrostatics('static.dat', body)


INFO:capytaine.bodies.bodies:Clipping buoy6.dat with respect to Plane(normal=[0. 0. 1.], point=[0. 0. 0.])
INFO:capytaine.meshes.clipper:Clipping half_of_buoy6.dat by Plane(normal=[0. 0. 1.], point=[0. 0. 0.]): no action.
INFO:capytaine.meshes.clipper:Clipping mirrored_of_half_of_buoy6.dat by Plane(normal=[0. 0. 1.], point=[0. 0. 0.]): no action.
             You might be overwriting existing files!


In [12]:
from numpy import infty
problem = cpt.RadiationProblem(body=body, radiating_dof="Heave", g=9.81, rho=1025, period = 8.0)


Check the position of the free_surface and the water_depth
or use body.keep_immersed_part() to clip the mesh.


In [20]:
diffraction_problem = cpt.DiffractionProblem(body=body, wave_direction=np.pi/2, omega=1.0)

In [21]:
solver = cpt.BEMSolver()
result = solver.solve(problem)

INFO:capytaine.bem.solver:Solve RadiationProblem(body=cylinder1.dat, period=8.000, water_depth=inf, radiating_dof='Heave', rho=1025.0).


In [22]:
all_radiation_problems = [cpt.RadiationProblem(body=body, radiating_dof=dof, omega=1.0) for dof in body.dofs]
diffraction_result = solver.solve(diffraction_problem)
all_radiation_results = solver.solve_all(all_radiation_problems)

INFO:capytaine.bem.solver:Solve DiffractionProblem(body=cylinder1.dat, omega=1.000, water_depth=inf, wave_direction=1.571).
INFO:capytaine.bem.solver:Solve RadiationProblem(body=cylinder1.dat, omega=1.000, water_depth=inf, radiating_dof='Surge').
INFO:capytaine.bem.solver:Solve RadiationProblem(body=cylinder1.dat, omega=1.000, water_depth=inf, radiating_dof='Sway').
INFO:capytaine.bem.solver:Solve RadiationProblem(body=cylinder1.dat, omega=1.000, water_depth=inf, radiating_dof='Heave').
INFO:capytaine.bem.solver:Solve RadiationProblem(body=cylinder1.dat, omega=1.000, water_depth=inf, radiating_dof='Roll').
INFO:capytaine.bem.solver:Solve RadiationProblem(body=cylinder1.dat, omega=1.000, water_depth=inf, radiating_dof='Pitch').
INFO:capytaine.bem.solver:Solve RadiationProblem(body=cylinder1.dat, omega=1.000, water_depth=inf, radiating_dof='Yaw').


In [10]:
dataset = cpt.assemble_dataset([diffraction_result] + all_radiation_results)

In [11]:
from capytaine.io.xarray import separate_complex_values
separate_complex_values(dataset).to_netcdf("dataset.nc")